This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [3]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# First, import the relevant modules
import requests
import json
import pandas as pd

In [5]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key=' + API_KEY
r = requests.get(url)

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset_data': {'limit': 1, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2000-06-07', 'end_date': '2020-12-01', 'frequency': 'daily', 'data': [['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [7]:
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY
data2017 =  requests.get(url)
r = data2017.json()

I chose to work with the data as a dataframe. It seemed the easiest way to handle this dataset. I felt it was a cleanest way to gather the data and looking at how other cohorts worked with the data, was less confusing. I was concerned after trying to work with it as just straight data I would not be able to remove empty values which was was finding previously with the corresponding date. 

In [8]:
test = pd.DataFrame(r['dataset_data']['data'])

In [10]:
test.columns = r['dataset_data']['column_names']

In [12]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255 entries, 0 to 254
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   255 non-null    object 
 1   Open                   252 non-null    float64
 2   High                   255 non-null    float64
 3   Low                    255 non-null    float64
 4   Close                  255 non-null    float64
 5   Change                 1 non-null      float64
 6   Traded Volume          255 non-null    float64
 7   Turnover               255 non-null    float64
 8   Last Price of the Day  0 non-null      object 
 9   Daily Traded Units     0 non-null      object 
 10  Daily Turnover         0 non-null      object 
dtypes: float64(7), object(4)
memory usage: 22.0+ KB


In [18]:
test.head()

,Date,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
0,2017-12-29,51.76,51.94,51.45,51.76,0.49,34640.0,1792304.0,None,None,None
1,2017-12-28,51.65,51.82,51.43,51.60,0.39,40660.0,2099024.0,None,None,None
2,2017-12-27,51.45,51.89,50.76,51.82,1.13,57452.0,2957018.0,None,None,None
3,2017-12-22,51.05,51.50,50.92,51.32,0.58,71165.0,3641949.0,None,None,None
4,2017-12-21,51.16,51.52,50.90,51.40,0.62,120649.0,6179433.0,None,None,None


I will need to know the change for the day and since the Change column of the dataframe is mainly empty, I chose to fill that column with the differece for each day. Since a stock could plummet it is not sufficient to only look at the maximum change but also the minimum change in the day as a stock could plummet and thus, create a larger net negative change.

In [40]:
test['Change'] = test['High']-test['Low']

In [41]:
print('The highest positive change was $' + str(max(test['Change'])))
print('The highest negative change was $' + str(min(test['Change'])))

The highest positive change was $2.8100000000000023
The highest negative change was $0.18999999999999773


In this sense case the highest daily change happens to be the positive of $2.81. Next is to calculate the Max and Min for the year.

In [17]:
print('The highest opening price was $' + str(max(test['Open'])))
print('The lowest opening price was $' + str(min(test['Open'])))

The highest opening price was $53.11
The lowest opening price was $34.0


To find the average daily trading volume during 2017, I did the following.

In [21]:
print('The average volume traded for 2017 was ' + str(test['Traded Volume'].mean()))

The average volume traded for 2017 was 89124.33725490196


In [22]:
print('The median volume traded for 2017 was ' + str(test['Traded Volume'].median()))

The median volume traded for 2017 was 76286.0


What was the largest change between any two days (based on Closing Price)? This I will most likely need to write a loop. 

In [31]:
change_in_closing = []
close = list(test['Close'])
print(close)

[51.76, 51.6, 51.82, 51.32, 51.4, 51.27, 51.66, 52.62, 52.01, 52.67, 53.09, 52.43, 52.14, 52.12, 51.47, 50.89, 51.25, 51.14, 49.86, 49.7, 48.75, 49.25, 49.2, 49.11, 49.2, 48.8, 48.39, 47.04, 46.84, 47.03, 47.05, 48.0, 48.34, 48.34, 47.21, 47.47, 46.26, 45.99, 45.97, 45.27, 46.04, 45.76, 46.41, 45.41, 45.0, 44.85, 45.0, 44.87, 45.0, 45.77, 45.96, 45.55, 45.4, 45.43, 45.29, 45.84, 45.74, 46.0, 46.05, 46.11, 44.98, 44.17, 43.56, 42.04, 42.37, 42.07, 42.06, 41.46, 41.99, 42.44, 42.27, 42.42, 42.52, 42.45, 42.73, 42.85, 42.67, 42.77, 42.55, 42.62, 42.6, 42.41, 41.9, 41.94, 41.85, 41.91, 42.14, 42.69, 42.71, 42.71, 42.26, 42.41, 42.5, 42.05, 42.28, 42.3, 41.94, 41.68, 41.81, 44.37, 44.96, 45.07, 44.97, 45.56, 45.45, 45.3, 44.97, 45.25, 45.16, 44.82, 44.61, 45.44, 45.66, 45.57, 45.0, 45.6, 45.53, 44.95, 44.95, 44.2, 44.7, 44.53, 44.62, 45.19, 44.8, 45.75, 45.44, 45.2, 45.68, 46.83, 47.21, 46.99, 47.29, 46.99, 47.37, 46.64, 46.63, 45.67, 46.33, 46.32, 46.31, 47.44, 46.27, 47.43, 47.43, 46.99, 

In [35]:
for x in range(len(test['Close'])-1):
    change_in_closing.append(close[x]-close[x+1])

In [38]:
print('The largest positive change between two days was $' + str(max(change_in_closing)))
print('The largest negative change between two days was $' + str(min(change_in_closing)))

The largest positive change between two days was $1.7199999999999989
The largest negative change between two days was $-2.559999999999995


The largest change between two days based on the closing price was -$2.56. 